In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import binom, poisson

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#reading the data
df=pd.read_csv('/content/drive/MyDrive/property.csv')

In [ ]:
#Data overview
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [ ]:
df.isnull().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64

**Q1**.Use the dataset named `property.csv` provided to you. This contains data on real estate prices in Australia. Now use this data to test the following hypothesis:

For the suburb Altona, it is postulated that a typical property sells for `$800,000.` Use the data at hand to test this assumption. Is the typical property price really `$800,000` or has it increased?

In [ ]:
#H_0: The property price is more than $800000 or above
#H_1: The property price is less than $800000

In [ ]:
# perform one sample t-test
altona_data = df[df['Suburb'] == 'Altona']['Price']
assumed_price = 800000
significance_level = 0.05
t_stat, p_value = stats.ttest_1samp(altona_data, assumed_price)
print(t_stat,p_value)

if p_value < significance_level:
    print("The typical property price is different from $800,000 (p < 0.05).")
else:
    print("There is no significant difference in the typical property price from $800,000 (p >= 0.05).")

1.0277020770199676 0.307483271305555
There is no significant difference in the typical property price from $800,000 (p >= 0.05).


2. For the year 2016, is there any difference in prices of properties sold in the summer months
vs winter months? Consider months from October till March as winter months and rest as
summer months. Use a significance level of 5%

In [ ]:
#Converting the date column in date format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [ ]:
winter_months = [10, 11, 12, 1, 2, 3]

df['Season'] = np.where(df['Date'].dt.month.isin(winter_months), 'Winter', 'Summer')

summer_data = df[(df['Season'] == 'Summer') & (df['Date'].dt.year == 2016)]['Price']
winter_data = df[(df['Season'] == 'Winter') & (df['Date'].dt.year == 2016)]['Price']

t_stat, p_value = stats.ttest_ind(winter_data,summer_data)
print(t_stat,p_value)

if p_value < significance_level:
    print("There is a significant difference in property prices between summer and winter months in 2016 (p < 0.05).")
else:
    print("There is no significant difference in property prices between summer and winter months in 2016 (p >= 0.05).")

4.043386317851058 5.3309767667631686e-05
There is a significant difference in property prices between summer and winter months in 2016 (p < 0.05).


3. For the suburb Abbotsford, what is the probability that out of 10 properties sold, 3 will not
have a car parking? Use the column car in the dataset. Round off your answer to 3 decimal
places

In [ ]:
#Filtering the data for the suburb Abbotsford
ab_data = df[df['Suburb'] == 'Abbotsford']

In [ ]:
# Number of properties not having a car parking
no_park_count = ab_data['Car'].value_counts()[0]

In [ ]:
#Total number of properties in suburb Abbotsford
Total_properties_count = ab_data.shape[0]

In [ ]:
#probability of success ie not having a  car parking
p = no_park_count / Total_properties_count
print(p)

0.26785714285714285


In [ ]:
num_successes = 3
num_trials=10
prob_no_car=binom.pmf(num_successes,num_trials,p)
print("Probability of 3 out of 10 properties in Abbotsford not having car parking:", round(prob_no_car, 3))

Probability of 3 out of 10 properties in Abbotsford not having car parking: 0.26


4. In the suburb Abbotsford, what are the chances of finding a property with 3 rooms? Round
your answer to 3 decimal places.

In [ ]:
ab_data['Rooms'].value_counts()

2    27
3    20
1     5
4     4
Name: Rooms, dtype: int64

In [ ]:
# Number of properties with 3 rooms
count_3_room = ab_data['Rooms'].value_counts()[3]

20

In [ ]:
prob_3_rooms = count_3_room / Total_properties_count
print("Probability of finding a property with 3 rooms in Abbotsford:", round(prob_3_rooms, 3))

Probability of finding a property with 3 rooms in Abbotsford: 0.357


5. In the suburb Abbotsford, what are the chances of finding a property with 2 bathrooms?
Round your answer to 3 decimal places.

In [ ]:
ab_data['Bathroom'].value_counts()

1.0    35
2.0    19
3.0     2
Name: Bathroom, dtype: int64

In [ ]:
# Number of properties with 2 bathrooms
count_2_bathroom = ab_data['Bathroom'].value_counts()[2]

19

In [ ]:
prob_2_bath=count_2_bathroom / Total_properties_count
print("Probability of finding a property with 2 bathrooms in Abbotsford:", round(prob_2_bath, 3))

Probability of finding a property with 2 bathrooms in Abbotsford: 0.339
